# 230911_웹페이지 크롤링하기

## 안산교육지원청 초등교육과 페이지 크롤링 하기

In [10]:
from bs4 import BeautifulSoup
import requests

# 웹페이지 URL
url = 'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType=A'

# 웹페이지 요청 및 응답 받기
response = requests.get(url)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
divs = soup.find_all('div', {'class': 'office'})

for div in divs:
    # 각 섹션에서 테이블 찾기
    table = div.find('table', {'class': 'dtl'})
    
    # 테이블의 모든 행 찾기
    rows = table.find_all('tr')
    
    for row in rows:
        # 각 행에서 'td' 태그 찾기
        cols = row.find_all('td')
        
        # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
        if len(cols) >= 3:
            position = cols[0].text.strip()  # 직위
            name = cols[1].text.strip()  # 성명
            task = cols[2].text.strip()  # 담당업무

            print(f'직위: {position}, 성명: {name}, 담당업무: {task}')


직위: 초등교육지원과장(031-412-4510), 성명: 민윤, 담당업무: • 초등교육지원과 업무 총괄
직위: 장학사(031-412-4511), 성명: 전은진, 담당업무: • 초등 업무 기획 및 조정• 타 국, 부서와 업무 협의 및 조정• 안산 교육계획 수립에 관한 사항• 교육지원청 기관평가 운영에 관한 사항• 교육장 협의회에 관한 사항• 공모 교육장 평가에 관한 사항• 교육전문직원 장학행정협의회 운영에 관한 사항• 교(원)장 통합 협의회 운영• 초등 학교장 역량강화 및 워크숍 운영• 초등 교(원)감 통합 협의회 운영• 자율장학 계획 수립 및 운영에 관한 사항• 초등 지구장학협의회(교장․교감) 운영에 관한 사항• 교육장 학교방문 장학• 부서 간 소통협의회 운영• 선도교육지원청 관련 팀장 통합회의 운영
직위: 장학사(031-412-4514), 성명: 진혜련, 담당업무: • 초등 미래교육 운영에 관한 사항• 초등 교육전문직원 직무역량 평가 운영에 관한 사항• 학교안 전문적학습공동체 교원 역량강화• 전문직 전문적학습공동체 운영• 초등 지역 맞춤형 연수(지역 기반 거점연수) 운영 -성장단계별 초등 교원 역량 강화 프로그램 운영• 초등 교육활동보호 및 교권에 관한 사항• 학생 7대 안전교육과정(나침반 5분 안전교육 포함)    운영• 학교안전관리위원회 운영에 관한 사항• 초등 교원능력개발평가 운영에 관한 사항• 초등 담임장학 기획 및 운영에 관한 사항• 선도교육지원청 담임장학 아카이브 구축• 초등교사 연구년제 및 신규교사 연수 운영• 초등 장학 네트워크 운영에 관한 사항• 초등 학교평가 운영에 관한 사항
직위: 주무관(031-412-4512), 성명: 한송이, 담당업무: • 초등과 회계 업무 총괄• 업무추진비(국,과장) 운영에 관한 사항• 초등 미래교육 운영에 관한 사항• 초등 자율학교 지정‧신청에 관한 사항• 학교 안 전문적 학습공동체 직무연수 운영에 관한 사항• 안산교육계획 현장 평가(온라인 설문)에 관한 사항• 사회복무요원 복무 관리• 연말 유공 교원 및 기관 표

## 크롤링 결과물 데이터베이스에 저장하기

In [11]:
from bs4 import BeautifulSoup
import requests
import sqlite3

# 웹페이지 URL
url = 'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType=A'

# 웹페이지 요청 및 응답 받기
response = requests.get(url)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# SQLite 데이터베이스 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect('database.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성 (이미 테이블이 있는 경우 에러가 발생하므로 IF NOT EXISTS를 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        position TEXT,
        name TEXT,
        task TEXT
    )
''')

# 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
divs = soup.find_all('div', {'class': 'office'})

for div in divs:
    # 각 섹션에서 테이블 찾기
    table = div.find('table', {'class': 'dtl'})
    
    # 테이블의 모든 행 찾기
    rows = table.find_all('tr')
    
    for row in rows:
        # 각 행에서 'td' 태그 찾기
        cols = row.find_all('td')
        
        # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
        if len(cols) >= 3:
            position = cols[0].text.strip()  # 직위
            name = cols[1].text.strip()  # 성명
            task = cols[2].text.strip()  # 담당업무

            # 데이터베이스에 데이터 삽입
            cursor.execute('''
                INSERT INTO employees (position, name, task) 
                VALUES (?, ?, ?)
            ''', (position, name, task))

# 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()


## 직위와 연락처 분리하기

In [3]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re

# 웹페이지 URL
url = 'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType=A'

# 웹페이지 요청 및 응답 받기
response = requests.get(url)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# SQLite 데이터베이스 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect('database_up.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성 (이미 테이블이 있는 경우 에러가 발생하므로 IF NOT EXISTS를 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        position TEXT,
        name TEXT,
        task TEXT,
        contact TEXT
    )
''')

# 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
divs = soup.find_all('div', {'class': 'office'})

for div in divs:
    # 각 섹션에서 테이블 찾기
    table = div.find('table', {'class': 'dtl'})
    
    # 테이블의 모든 행 찾기
    rows = table.find_all('tr')
    
    for row in rows:
        # 각 행에서 'td' 태그 찾기
        cols = row.find_all('td')
        
        # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
        if len(cols) >= 3:
            position_contact = cols[0].text.strip()  # 직위와 연락처 정보
            
            # 정규 표현식을 사용하여 직위와 연락처 분리
            match = re.match(r'(.*)(\(.+\))', position_contact)
            if match:
                position = match.group(1).strip()  # 직위
                contact = match.group(2).strip()  # 연락처
            else:
                position = position_contact
                contact = ''

            name = cols[1].text.strip()  # 성명
            task = cols[2].text.strip()  # 담당업무

            # 데이터베이스에 데이터 삽입
            cursor.execute('''
                INSERT INTO employees (position, name, task, contact) 
                VALUES (?, ?, ?, ?)
            ''', (position, name, task, contact))

# 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()


## 안산교육지원청 모든 부서 크롤링하기

In [4]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re

# SQLite 데이터베이스 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect('ansan_database.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성 (이미 테이블이 있는 경우 에러가 발생하므로 IF NOT EXISTS를 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        position TEXT,
        name TEXT,
        task TEXT,
        contact TEXT
    )
''')

# 'A'부터 'I'까지 반복
for org_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    # 웹페이지 URL
    url = f'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType={org_type}'

    # 웹페이지 요청 및 응답 받기
    response = requests.get(url)

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
    divs = soup.find_all('div', {'class': 'office'})

    for div in divs:
        # 각 섹션에서 테이블 찾기
        table = div.find('table', {'class': 'dtl'})
        
        # 테이블의 모든 행 찾기
        rows = table.find_all('tr')
        
        for row in rows:
            # 각 행에서 'td' 태그 찾기
            cols = row.find_all('td')
            
            # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
            if len(cols) >= 3:
                position_contact = cols[0].text.strip()  # 직위와 연락처 정보
                
                # 정규 표현식을 사용하여 직위와 연락처 분리
                match = re.match(r'(.*)(\(.+\))', position_contact)
                if match:
                    position = match.group(1).strip()  # 직위
                    contact = match.group(2).strip()  # 연락처
                else:
                    position = position_contact
                    contact = ''

                name = cols[1].text.strip()  # 성명
                task = cols[2].text.strip()  # 담당업무

                # 데이터베이스에 데이터 삽입
                cursor.execute('''
                    INSERT INTO employees (position, name, task, contact) 
                    VALUES (?, ?, ?, ?)
                ''', (position, name, task, contact))

# 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()


## 과와 부서 입력하기

In [5]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re

# SQLite 데이터베이스 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect('database.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성 (이미 테이블이 있는 경우 에러가 발생하므로 IF NOT EXISTS를 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        department TEXT,
        section TEXT,
        position TEXT,
        name TEXT,
        contact TEXT,
        task TEXT
    )
''')

# 'A'부터 'I'까지 반복
for org_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    # 웹페이지 URL
    url = f'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType={org_type}'

    # 웹페이지 요청 및 응답 받기
    response = requests.get(url)

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 과와 부서 정보 찾기
    department = soup.find('h4', {'class': 'tit'}).text.strip()
    section = soup.find('h4', {'class': 'tit2'}).text.strip()

    # 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
    divs = soup.find_all('div', {'class': 'office'})

    for div in divs:
        # 각 섹션에서 테이블 찾기
        table = div.find('table', {'class': 'dtl'})
        
        # 테이블의 모든 행 찾기
        rows = table.find_all('tr')
        
        for row in rows:
            # 각 행에서 'td' 태그 찾기
            cols = row.find_all('td')
            
            # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
            if len(cols) >= 3:
                position_contact = cols[0].text.strip()  # 직위와 연락처 정보
                
                # 정규 표현식을 사용하여 직위와 연락처 분리
                match = re.match(r'(.*)(\(.+\))', position_contact)
                if match:
                    position = match.group(1).strip()  # 직위
                    contact = match.group(2).strip()  # 연락처
                else:
                    position = position_contact
                    contact = ''

                name = cols[1].text.strip()  # 성명
                task = cols[2].text.strip()  # 담당업무

                # 데이터베이스에 데이터 삽입
                cursor.execute('''
                    INSERT INTO employees (department, section, position, name, contact, task) 
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (department, section, position, name, contact, task))

# 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()


## 타임스탬프로 구분하기

In [ ]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re
from datetime import datetime

# SQLite 데이터베이스 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect('database.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성 (이미 테이블이 있는 경우 에러가 발생하므로 IF NOT EXISTS를 사용)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        department TEXT,
        section TEXT,
        position TEXT,
        name TEXT,
        contact TEXT,
        task TEXT,
        timestamp TEXT
    )
''')

# 'A'부터 'I'까지 반복
for org_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    # 웹페이지 URL
    url = f'https://www.goeas.kr/USR/ORG/MNU6/OrgDetail.do?orgType={org_type}'

    # 웹페이지 요청 및 응답 받기
    response = requests.get(url)

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 과와 부서 정보 찾기
    department = soup.find('h4', {'class': 'tit'}).text.strip()
    section = soup.find('h4', {'class': 'tit2'}).text.strip()

    # 현재 시간을 타임스탬프로 저장
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 'div' 태그와 'class' 속성이 'office'인 모든 섹션 찾기
    divs = soup.find_all('div', {'class': 'office'})

    for div in divs:
        # 각 섹션에서 테이블 찾기
        table = div.find('table', {'class': 'dtl'})
        
        # 테이블의 모든 행 찾기
        rows = table.find_all('tr')
        
        for row in rows:
            # 각 행에서 'td' 태그 찾기
            cols = row.find_all('td')
            
            # 'td' 태그가 3개 이상 있는 경우 (직위, 성명, 담당업무 정보가 있는 경우)
            if len(cols) >= 3:
                position_contact = cols[0].text.strip()  # 직위와 연락처 정보
                
                # 정규 표현식을 사용하여 직위와 연락처 분리
                match = re.match(r'(.*)(\(.+\))', position_contact)
                if match:
                    position = match.group(1).strip()  # 직위
                    contact = match.group(2).strip()  # 연락처
                else:
                    position = position_contact
                    contact = ''

                name = cols[1].text.strip()  # 성명
                task = cols[2].text.strip()  # 담당업무

                # 데이터베이스에 데이터 삽입
                cursor.execute('''
                    INSERT INTO employees (department, section, position, name, contact, task, timestamp) 
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (department, section, position, name, contact, task, timestamp))

# 변경사항 저장
conn.commit()

# 데이터베이스 연결 종료
conn.close()
